
    CREATE TABLE IF NOT EXISTS code_completion 
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        note_id INTEGER,
        dataframe_header JSON,
        solution_dataframe JSON,
        problem_description TEXT,
        dataset_path TEXT,
        FOREIGN KEY (note_id) REFERENCES notes (id)           


In [1]:
import sqlite3

def write_example_to_db(dataframe_head, dataframe_target, dataset_path, description):
    conn = sqlite3.connect("../../flashcards.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO notes DEFAULT VALUES")
    note_id = cursor.lastrowid
    cursor.execute(
        "INSERT INTO code_completion (note_id, dataframe_header, solution_dataframe, problem_description, dataset_path) VALUES (?, ?, ?, ?, ?)",
        (note_id, dataframe_head.to_json(), dataframe_target.to_json(), description, dataset_path),
    )
    conn.commit()
    conn.close()


In [1]:
import polars as pl
import pandas as pd 


dataset_path = "backend/code_completion/data/contoso_sales.csv"

df = pl.read_csv("../../"+dataset_path)

# transformation
result = (
    df
    .group_by('Brand')
    .agg(pl.col('Quantity').sum().alias('Sum of Quantity'))
)

input_header = pd.DataFrame(df.head(3).to_dict())

result = pd.DataFrame(result.to_dict())

description = "Summarise the brand column by taking the sum of Quanaity, naming the new column 'Sum of Quanity'"
# 
#write_example_to_db(input_header, result, dataset_path, description)

In [13]:
def conv_to_dataframe(x):
    if type(x) != pl.DataFrame:
        return pl.DataFrame([x])
    return x 

# Chp3 examples 
dataset_path = "backend/code_completion/data/covid_19_deaths.csv"
df = pl.read_csv("../../"+dataset_path)
input_header = pd.DataFrame(df.head(3).to_dict())
result = df.is_duplicated().sum()
result = conv_to_dataframe(result)
result

column_0
i64
0


In [17]:
# counting unique for each 
description = 'count unique of each column'
result = df.select(pl.all().n_unique())
write_example_to_db(input_header, result, dataset_path, description)

Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
1,45,45,3,5,13,54,3,17,3537,10307,3556,2533,493,4264,2


In [19]:
description = 'Drop all non-unique rows for combination of columns "Year" and "COVID-19 Deaths"'
result = df.filter(df.select(['Year', 'COVID-19 Deaths']).is_unique())
result

Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""All Ages""",1146774,12303399,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""Under 1 year""",519,73213,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""0-17 years""",1696,130970,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""1-4 years""",285,14299,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""5-14 years""",509,22008,818,143,219,1390,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""09/27/2023""","""12/01/2020""","""12/31/2020""","""By Month""","""2020""","""12""","""Wisconsin""","""Female""","""All Ages""",876,3235,335,246,null,965,"""One or more data cells have co…"
"""09/27/2023""","""12/01/2021""","""12/31/2021""","""By Month""","""2021""","""12""","""Wisconsin""","""All Sexes""","""All Ages""",1341,6468,813,569,null,1589,"""One or more data cells have co…"
"""09/27/2023""","""01/01/2022""","""01/31/2022""","""By Month""","""2022""","""1""","""Wisconsin""","""All Sexes""","""All Ages""",1476,6746,766,550,16,1705,null


In [25]:
q1 = pl.col('COVID-19 Deaths').quantile(0.25)
q3 = pl.col('COVID-19 Deaths').quantile(0.75)
iqr = q3 - q1
threshold = 1.5
lower_limit = q1 - iqr * threshold
upper_limit = q3 + iqr * threshold

description = "Apply IQR filter with a threshold of 1.5 based on 'COVID-19 Deaths' to dataframe"
is_outlier_iqr = (pl.col('COVID-19 Deaths') < lower_limit) | (pl.col('COVID-19 Deaths') > upper_limit)
df_iqr_outlier_removed = (
    df
    .filter(is_outlier_iqr.not_())
)
df_iqr_outlier_removed.filter(is_outlier_iqr)
result

Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""All Ages""",1146774,12303399,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""Under 1 year""",519,73213,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""0-17 years""",1696,130970,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""1-4 years""",285,14299,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""5-14 years""",509,22008,818,143,219,1390,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""09/27/2023""","""07/01/2022""","""07/31/2022""","""By Month""","""2022""","""7""","""Puerto Rico""","""All Sexes""","""All Ages""",204,2982,408,133,null,481,"""One or more data cells have co…"
"""09/27/2023""","""08/01/2022""","""08/31/2022""","""By Month""","""2022""","""8""","""Puerto Rico""","""All Sexes""","""All Ages""",225,3002,437,137,null,531,"""One or more data cells have co…"
"""09/27/2023""","""09/01/2022""","""09/30/2022""","""By Month""","""2022""","""9""","""Puerto Rico""","""All Sexes""","""All Ages""",169,2841,401,118,null,457,"""One or more data cells have co…"


In [27]:
q1 = pl.col('COVID-19 Deaths').quantile(0.25)
q3 = pl.col('COVID-19 Deaths').quantile(0.75)
iqr = q3 - q1
threshold = 1.5
lower_limit = q1 - iqr * threshold
upper_limit = q3 + iqr * threshold

description = "Apply IQR filter to replace outliers with median with a threshold of 1.5 based on 'COVID-19 Deaths' to dataframe"
is_outlier_iqr = (pl.col('COVID-19 Deaths') < lower_limit) | (pl.col('COVID-19 Deaths') > upper_limit)
df_iqr_outlier_replaced = (
    df
    .with_columns(
        pl.when(is_outlier_iqr)
        .then(pl.col('COVID-19 Deaths').median())
        .otherwise(pl.col('COVID-19 Deaths'))
        .alias('COVID-19 Deaths')
    )
)
df_iqr_outlier_replaced

Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,str,str,str,str,f64,i64,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""All Ages""",0.0,12303399,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""Under 1 year""",0.0,73213,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""0-17 years""",0.0,130970,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""1-4 years""",0.0,14299,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""5-14 years""",0.0,22008,818,143,219,1390,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""","""9""","""Puerto Rico""","""Female""","""50-64 years""",null,75,14,null,0,14,"""One or more data cells have co…"
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""","""9""","""Puerto Rico""","""Female""","""55-64 years""",0.0,65,10,0,0,10,null
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""","""9""","""Puerto Rico""","""Female""","""65-74 years""",null,91,null,null,0,null,"""One or more data cells have co…"
